In [1]:
from arcgis.gis import GIS
from data_engineering.utils import fetch_charging_stations, fetch_hotcold_features, fetch_traffic_accidents
from urban_traffic.utils import fetch_traffic_data, filter_commute_cars, generate_car_renderer, prepare_traffic, read_traffic_features
import os
import pandas as pd

### Additional dependencies
We load the necessary variables using `dotenv`.  
Please, ensure that you setup before: `pip install python-dotenv`  
Furthermore, put a `.env` file into the root of this repository.

In [2]:
from dotenv import load_dotenv

def load_vars():
    load_dotenv(dotenv_path="../../../../.env", override=True)
    return os.getenv("ARCGIS_API_KEY"), os.getenv("TRAFFIC_DATA_FILE"), os.getenv("TRAFFIC_FEATURES")

In [3]:
api_key, traffic_data_filepath, traffic_features_filepath = load_vars()

### Connect to your GIS portal

In [4]:
gis = GIS(api_key=api_key)

### Use a map for the area of interest

In [5]:
def create_map(gis: GIS, location: str = "Ludwig-Erhard-Anlage 1, 60327 Frankfurt am Main, Germany"):
    map_view = gis.map(location)
    map_view.basemap.basemap = "osm"
    map_view.zoom = 17
    return map_view

In [6]:
map_view = create_map(gis)
map_view

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 962882.1493646716, 'ymin': 6465580.1136933…

### Spatial filter using the area of interest

In [7]:
distance_in_meters = 150
lon, lat = map_view.center[:2]
traffic_features = read_traffic_features(traffic_features_filepath, lon=lon, lat=lat, meters=distance_in_meters)
traffic_features[["trip", "person", "vehicle_type"]]

,trip,person,vehicle_type
0,12069,7524,Bike
1,12069,7524,Bike
2,12069,7524,Bike
3,12069,7524,Bike
4,12069,7524,Bike
...,...,...,...
9026,6263,3892,None
9027,6263,3892,None
9028,6263,3892,None
9029,6263,3892,None


In [8]:
traffic_features.dtypes

OBJECTID           int64
SHAPE           geometry
trip               int64
person             int64
vehicle_type      object
trip_time         object
dtype: object

In [9]:
prepared_traffic_features = prepare_traffic(traffic_features.copy())
prepared_traffic_features

,OBJECTID,SHAPE,trip,person,hour,minute,second,bike,car,pedestrian
0,229354,"{""x"": 8.650534751000066, ""y"": 50.1136668510000...",12069,7524,3,50,58,1,0,0
1,229379,"{""x"": 8.650534005000054, ""y"": 50.1136312120000...",12069,7524,3,50,59,1,0,0
2,229404,"{""x"": 8.650538966000056, ""y"": 50.1135955960000...",12069,7524,3,51,0,1,0,0
3,229429,"{""x"": 8.650543927000058, ""y"": 50.1135599790000...",12069,7524,3,51,1,1,0,0
4,229454,"{""x"": 8.650556937000033, ""y"": 50.1135254140000...",12069,7524,3,51,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...
9026,17190362,"{""x"": 8.652244763000056, ""y"": 50.1132610030000...",6263,3892,23,8,10,0,0,1
9027,17190390,"{""x"": 8.652244763000056, ""y"": 50.1132610030000...",6263,3892,23,8,11,0,0,1
9028,17190418,"{""x"": 8.652244763000056, ""y"": 50.1132610030000...",6263,3892,23,8,12,0,0,1
9029,17190446,"{""x"": 8.652244763000056, ""y"": 50.1132610030000...",6263,3892,23,8,13,0,0,1


In [10]:
prepared_traffic_features.dtypes

OBJECTID         int64
SHAPE         geometry
trip             int64
person           int64
hour             int32
minute           int32
second           int32
bike             int32
car              int32
pedestrian       int32
dtype: object

### Visualize commute traffic

In [11]:
commute_cars = filter_commute_cars(prepared_traffic_features.copy())
commute_cars

,OBJECTID,SHAPE,trip,person,hour,minute,second,bike,car,pedestrian
1040,2476833,"{""x"": 8.650609811000038, ""y"": 50.1136165960000...",4772,2962,8,5,37,0,1,0
1041,2476877,"{""x"": 8.650567598000066, ""y"": 50.1136170170000...",11422,7117,8,5,37,0,1,0
1042,2477413,"{""x"": 8.650652800000046, ""y"": 50.1134959590000...",4772,2962,8,5,38,0,1,0
1043,2477481,"{""x"": 8.650606480000022, ""y"": 50.1134955600000...",11422,7117,8,5,38,0,1,0
1044,2477874,"{""x"": 8.650717396000061, ""y"": 50.1133933380000...",11422,7117,8,5,39,0,1,0
...,...,...,...,...,...,...,...,...,...,...
4053,7116251,"{""x"": 8.65206020900007, ""y"": 50.11311437500006...",10887,6787,9,59,50,0,1,0
4054,7116948,"{""x"": 8.652221206000036, ""y"": 50.1130443840000...",10887,6787,9,59,51,0,1,0
4055,7117645,"{""x"": 8.652382205000038, ""y"": 50.1129743940000...",10887,6787,9,59,52,0,1,0
4056,7118341,"{""x"": 8.65254320400004, ""y"": 50.11290440200003...",10887,6787,9,59,53,0,1,0


In [12]:
commute_map_view = create_map(gis)
commute_cars.spatial.plot(commute_map_view, renderer=generate_car_renderer())
commute_map_view.zoom_to_layer(commute_cars)
commute_map_view

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 962783.3385332164, 'ymin': 6465530.3265399…

## Analyse geospatial patterns

In [14]:
hotcold_spots, drawing_info = fetch_hotcold_features(gis, commute_cars)
commute_map_view.content.add(hotcold_spots, drawing_info=drawing_info, options={"opacity": 0.3})

In [49]:
import arcpy